# Simple shear

In [93]:
import numpy as np
import pandas as pd
from scipy.linalg import eigh
from scipy.linalg import sqrtm
from scipy.linalg import polar
from scipy.linalg import inv

##### helper functions

In [97]:
def reconstruct_strain_matrix(eid, frame):
    exx = df_strain.loc[frame, "Exx_"+str(eid)]
    eyy = df_strain.loc[frame, "Eyy_"+str(eid)]
    ezz = df_strain.loc[frame, "Ezz_"+str(eid)]
    exy = df_strain.loc[frame, "Exy_"+str(eid)]
    eyz = df_strain.loc[frame, "Eyz_"+str(eid)]
    ezx = df_strain.loc[frame, "Ezx_"+str(eid)]
    e_mat = np.array([[exx, exy, ezx],
                      [exy, eyy, eyz],
                      [ezx, eyz, ezz]])
    return e_mat

In [98]:
def compute_angle_between_3d_points(a,b,c):
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return angle

### Wikipedia "Deformationsgradient"   
https://de.wikipedia.org/wiki/Deformationsgradient#Beispiel

In [84]:
tan_alpha = 5/6

In [85]:
def_F = np.array([[1., tan_alpha],
                [0., 1.]])
def_C = np.matmul(def_F.T, def_F)
w, v = eigh(def_C)

In [86]:
print("Eigenvalues:")
print(w[0])
print(w[1])

Eigenvalues:
0.44444444444444453
2.25


In [87]:
# get true strain values
np.log(w)/2

array([-0.40546511,  0.40546511])

In [88]:
v1 = v[:,0]
v2 = v[:,1]
tmp_22 = np.array([v1*w[0], v2*w[1]])
tmp_11 = np.array([v1, v2]).T

In [89]:
np.matmul(tmp_11, tmp_22)

array([[1.        , 0.83333333],
       [0.83333333, 1.69444444]])

In [90]:
sqrtm(np.matmul(tmp_11, tmp_22))

array([[0.92307692, 0.38461538],
       [0.38461538, 1.24358974]])

Result agrees with result on wikipedia.

In [51]:
np.array([[12/13, 5/13],
          [5/13, (97/6)*(1/13)]])

array([[0.92307692, 0.38461538],
       [0.38461538, 1.24358974]])

### Results from finite element simulation

In [107]:
# read results
df_nodes = pd.read_csv("df_nodes.csv")
df_strain = pd.read_csv("df_strain.csv")
end_frame = len(df_strain)-1

In [110]:
# check positions of corner nodes
tan_alpha =  df_nodes.loc[end_frame, "COORDx_15"] -df_nodes.loc[end_frame, "COORDx_11"]
height = df_nodes.loc[end_frame, "COORDy_15"] -df_nodes.loc[end_frame, "COORDy_11"]
print("tan_alpha in simulation: {:.4f} ".format(tan_alpha))
print("deformed height in simulation: {:.4f} ".format(height))

tan_alpha in simulation: 0.8327 
deformed height in simulation: 1.0000 


In [111]:
# (true) strain tensor from finite element results
eid = 1
e_mat = reconstruct_strain_matrix(eid, end_frame)
e_mat

array([[-0.3698638 ,  0.16354624,  0.        ],
       [ 0.16354624,  0.3698638 ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [112]:
w,v = np.linalg.eigh(e_mat)

In [113]:
v1 = v[:,0]
v2 = v[:,1]
v3 = v[:,2]
centroid = np.array([0., 0., 0.])
vertical = np.array([centroid[0], centroid[1]+10, centroid[2]])
theta_def = compute_angle_between_3d_points(vertical, centroid, v3)
np.rad2deg(theta_def)

11.927019252267872

In [114]:
angle = theta_def
rot_mat = np.array([[np.cos(angle), -np.sin(angle), 0.],
                    [np.sin(angle), np.cos(angle), 0.],
                    [0., 0., 1.]])
mult1 = np.matmul(rot_mat, e_mat)
eps_princ = np.matmul(mult1, rot_mat.T)

In [115]:
# true strain tensor in principal direction
eps_princ

array([[-4.04408953e-01, -7.48356412e-17,  0.00000000e+00],
       [-7.10288720e-17,  4.04408953e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [116]:
# stretch tensor U in principal direction
np.exp(eps_princ*2)

array([[0.44538424, 1.        , 1.        ],
       [1.        , 2.24525232, 1.        ],
       [1.        , 1.        , 1.        ]])

## How to recover deformation gradient from finite element result???   
Need to calcaulate tensor def_F from e_mat.